# Image Classification
Tutorial from https://hackernoon.com/training-an-image-classifier-from-scratch-in-15-minutes-3c140f5fa1af

In [3]:
import torchvision.transforms as tt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from fastai.dataset import ModelData
from fastai.conv_learner import ConvLearner

/N/u/sekanaya/anaconda3/envs/pynet/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/N/u/sekanaya/anaconda3/envs/pynet/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/N/u/sekanaya/anaconda3/envs/pynet/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
def get_data(bs, num_workers):
    PATH = "/N/u/sekanaya/sali/git/github/esaliya/python/data/cifar10/"
    trn_dir, val_dir = PATH + 'train', PATH + 'test'
    stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    
    # Data transforms (normalization & data augmentation)
    tfms = [tt.ToTensor(), tt.Normalize(*stats)]
    aug_tfms = tt.Compose([tt.RandomCrop(32, padding=4), 
                           tt.RandomHorizontalFlip()] + tfms)
    # PyTorch datasets
    trn_ds = ImageFolder(trn_dir, aug_tfms)
    val_ds = ImageFolder(val_dir, tt.Compose(tfms))
    aug_ds = ImageFolder(val_dir, aug_tfms)
    
    # PyTorch data loaders
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, 
                        num_workers=num_workers, pin_memory=True)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, 
                        num_workers=num_workers, pin_memory=True)
    aug_dl = DataLoader(aug_ds, batch_size=bs, shuffle=False, 
                        num_workers=num_workers, pin_memory=True)
    
    # FastAI model data 
    data = ModelData(PATH, trn_dl, val_dl)
    data.aug_dl = aug_dl
    data.sz = 32
    
    return data

In [5]:
def get_learner(arch, bs):
    """Create a FastAI learner using the given model"""
    data = get_data(bs, 24)
    learn = ConvLearner.from_model_data(arch.cuda(), data)
    learn.crit = nn.CrossEntropyLoss()
#     learn.metrics = ["accuracy"]
    return learn

def get_TTA_accuracy(learn):
    """Calculate accuracy with Test Time Agumentation(TTA)"""
    preds, targs = learn.TTA()
    preds = 0.6 * preds[0] + 0.4 * preds[1:].sum(0)
    return accuracy_np(preds, targs)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

def conv_2d(ni, nf, stride=1, ks=3):
    """3x3 convolution with 1 pixel padding"""
    return nn.Conv2d(in_channels=ni, out_channels=nf, 
                     kernel_size=ks, stride=stride, 
                     padding=ks//2, bias=False)

def bn_relu_conv(ni, nf):
    """BatchNorm → ReLU → Conv2D"""
    return nn.Sequential(nn.BatchNorm2d(ni), 
                         nn.ReLU(inplace=True), 
                         conv_2d(ni, nf))

class BasicBlock(nn.Module):
    """Residual block with shortcut connection"""
    def __init__(self, ni, nf, stride=1):
        super().__init__()
        self.bn = nn.BatchNorm2d(ni)
        self.conv1 = conv_2d(ni, nf, stride)
        self.conv2 = bn_relu_conv(nf, nf)
        self.shortcut = lambda x: x
        if ni != nf:
            self.shortcut = conv_2d(ni, nf, stride, 1)
    
    def forward(self, x):
        x = F.relu(self.bn(x), inplace=True)
        r = self.shortcut(x)
        x = self.conv1(x)
        x = self.conv2(x) * 0.2
        return x.add_(r)

In [7]:
def make_group(N, ni, nf, stride):
    """Group of residual blocks"""
    start = BasicBlock(ni, nf, stride)
    rest = [BasicBlock(nf, nf) for j in range(1, N)]
    return [start] + rest

class Flatten(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, x): return x.view(x.size(0), -1)

class WideResNet(nn.Module):
    def __init__(self, n_groups, N, n_classes, k=1, n_start=16):
        super().__init__()      
        # Increase channels to n_start using conv layer
        layers = [conv_2d(3, n_start)]
        n_channels = [n_start]
        
        # Add groups of BasicBlock(increase channels & downsample)
        for i in range(n_groups):
            n_channels.append(n_start*(2**i)*k)
            stride = 2 if i>0 else 1
            layers += make_group(N, n_channels[i], 
                                 n_channels[i+1], stride)
        
        # Pool, flatten & add linear layer for classification
        layers += [nn.BatchNorm2d(n_channels[3]), 
                   nn.ReLU(inplace=True), 
                   nn.AdaptiveAvgPool2d(1), 
                   Flatten(), 
                   nn.Linear(n_channels[3], n_classes)]
        
        self.features = nn.Sequential(*layers)
        
    def forward(self, x): return self.features(x)
    
def wrn_22(): 
    return WideResNet(n_groups=3, N=3, n_classes=10, k=6)

In [8]:
%%time
learn = get_learner(wrn_22(), 128)


CPU times: user 2.38 s, sys: 668 ms, total: 3.04 s
Wall time: 3.06 s


In [9]:
%%time
learn.clip = 1e-1
learn.fit(1.5, 1, wds=1e-4, cycle_len=20, use_clr_beta=(12, 15, 0.95, 0.85))

epoch      trn_loss   val_loss                              
    0      1.276207   1.528313  
    1      0.976014   1.166423                               
    2      0.81544    0.991873                               
    3      0.70645    0.96148                                
    4      0.65028    0.783206                               
    5      0.633292   1.216715                               
    6      0.615304   1.043887                               
    7      0.610193   1.211688                               
    8      0.608913   1.132347                               
    9      0.56096    0.751941                               
    10     0.553828   1.025181                               
    11     0.534848   0.898866                               
    12     0.512698   1.01549                                
    13     0.485222   0.712462                               
    14     0.43034    0.620588                               
    15     0.391012   0.579668        

In [11]:
get_TTA_accuracy(learn)

NameError: name 'accuracy_np' is not defined